# Ensemble Model Training - Simplified Single-Branch Models

This notebook trains separate simplified CNN models for each feature type and then creates an ensemble.

In [6]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Average
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from modules.PostgresDBHandler import PostgresDBHandler
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

In [7]:
# Configuration
dbParams = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres_server",
    "port": "5432",
}

EPOCHS = 200
BATCH_SIZE = 32
KFOLD_SPLITS = 5
FIXED_LENGTH = 128

# Feature types to train models for
FEATURE_TYPES = [
    'mel_spectrogram', 'mfcc', 'chromagram', 'spectral_contrast',
    'tonnetz', 'constant_q', 'cqt', 'stft', 'harmonic_percussive', 'onset_strength'
]

# GPU configuration
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Number of available GPUs: {len(gpus)}")
    except RuntimeError as e:
        print(e)

In [8]:
# Initialize database connection
db = PostgresDBHandler(**dbParams)
db.connect()

# Get instrument mappings
instruments_mappings = db.get_mappings_instruments()
num_classes = len(instruments_mappings)
print(f"Number of instrument classes: {num_classes}")
print("Instruments:", instruments_mappings['name'].tolist())

db.close()

Number of instrument classes: 9
Instruments: ['clarinet', 'flute', 'cello', 'violin', 'bass', 'sax', 'trumpet', 'oboe', 'piccolo']


In [9]:
dbConnect = PostgresDBHandler(**dbParams)
dbConnect.connect()
audioIDs = dbConnect.get_all_unique_audio_ids_in_processed()

all_processed_data = []
for audio_id in audioIDs:
    features = dbConnect.get_all_feature_types_for_audio(audio_id)
    feature_dict = {f['featureTypeName']: f['featurePath'] for f in features}
    instrumentID = dbConnect.get_audio_file(audio_id)['instrumentID']
    feature_dict['instrumentID'] = instrumentID
    all_processed_data.append(feature_dict)

dbConnect.close()

In [10]:
processed_df = pd.DataFrame(all_processed_data)
processed_df

,mel_spectrogram,mfcc,chromagram,spectral_contrast,tonnetz,constant_q,cqt,stft,harmonic_percussive,onset_strength,instrumentID
0,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/e6514fa9-9a...,ensemble_intermediate_results/chromagram/80e1a...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/22537895...,ensemble_intermediate_results/constant_q/2afeb...,ensemble_intermediate_results/cqt/dbd992d8-b60...,ensemble_intermediate_results/stft/6b51a84f-67...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/8...,9
1,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/b5fc5fc4-f1...,ensemble_intermediate_results/chromagram/8c17f...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/776160d0...,ensemble_intermediate_results/constant_q/c0433...,ensemble_intermediate_results/cqt/edcf8896-27a...,ensemble_intermediate_results/stft/d960a43b-83...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/e...,5
2,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/63e73b9a-32...,ensemble_intermediate_results/chromagram/a13f2...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/1d56a878...,ensemble_intermediate_results/constant_q/6112f...,ensemble_intermediate_results/cqt/8707d92b-ddc...,ensemble_intermediate_results/stft/eae2472c-08...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/b...,2
3,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/67edce83-82...,ensemble_intermediate_results/chromagram/2f5fe...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/15080b4c...,ensemble_intermediate_results/constant_q/9960d...,ensemble_intermediate_results/cqt/73feb1df-9a5...,ensemble_intermediate_results/stft/92432c38-d8...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/d...,1
4,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/a1dd9589-66...,ensemble_intermediate_results/chromagram/b4baf...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/423d88bf...,ensemble_intermediate_results/constant_q/1cf32...,ensemble_intermediate_results/cqt/99db3a2e-66b...,ensemble_intermediate_results/stft/eb536e2f-a3...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/3...,7
...,...,...,...,...,...,...,...,...,...,...,...
2154,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/6b76d345-dc...,ensemble_intermediate_results/chromagram/a722f...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/de0e87b2...,ensemble_intermediate_results/constant_q/aa54b...,ensemble_intermediate_results/cqt/41fad8b0-e41...,ensemble_intermediate_results/stft/b9cfe7ad-4d...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/7...,8
2155,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/351a6c33-c6...,ensemble_intermediate_results/chromagram/f4671...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/4dc827f2...,ensemble_intermediate_results/constant_q/2795c...,ensemble_intermediate_results/cqt/6dac953a-272...,ensemble_intermediate_results/stft/65821940-86...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/d...,3
2156,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/3a40ae3e-82...,ensemble_intermediate_results/chromagram/dd4dd...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/8f86a816...,ensemble_intermediate_results/consta

In [11]:
def get_input_shape(feature_type, df):
    feature_path_col = feature_type 
    for path in df[feature_path_col]:
        if isinstance(path, str) and os.path.exists(path):
            arr = np.load(path)
            return arr.shape
    raise ValueError(f"No valid file found for {feature_type}")

In [12]:
class SingleFeatureDataGenerator(Sequence):
    def __init__(self, df, feature_col, batch_size=32, shuffle=True, num_classes=None):
        self.df = df.reset_index(drop=True)
        self.feature_col = feature_col
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            try:
                arr = np.load(row[self.feature_col])
            except Exception as e:
                print(f"Error loading {row[self.feature_col]}: {e}")
                continue
        
            if np.isnan(arr).any() or np.isinf(arr).any():
                raise ValueError(f"Feature file {row[self.feature_col]} contains NaNs or Infs.")
        
            arr = (arr - np.mean(arr)) / (np.std(arr) + 1e-8)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, -1)  # shape: (H, W, 1)
        
            X.append(arr)
            y.append(row['instrumentID'])  # already label-encoded
        
        X = np.array(X)
        y = to_categorical(np.array(y), num_classes=self.num_classes)
        
        return X, y

In [13]:
def create_simple_model(input_shape, num_classes, model_name="simple_cnn"):
    input_layer = Input(shape=(*input_shape, 1), name=f"{model_name}_input")

    x = Conv2D(4, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)

    x = Flatten()(x)

    output = Dense(num_classes, activation='softmax', name=f"{model_name}_output")(x)

    model = Model(inputs=input_layer, outputs=output, name=model_name)
    return model

In [14]:
results = {}

for feature_type in tqdm(FEATURE_TYPES, desc="Training features"):
    print(f"\n{'='*40}\nTraining model for {feature_type}\n{'='*40}")

    feature_col = feature_type
    feature_df = processed_df.dropna(subset=[feature_col])
    
    # Global label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(feature_df['instrumentID'])

    feature_df = feature_df.copy()
    feature_df['instrumentID'] = label_encoder.transform(feature_df['instrumentID'])
    num_classes = len(label_encoder.classes_)
    input_shape = get_input_shape(feature_type, feature_df)


    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    accuracy_list, loss_list, history_list = [], [], []
    classification_reports, confusion_matrices = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(feature_df)):
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")
        train_df = feature_df.iloc[train_idx].reset_index(drop=True)
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)

        train_df, val_df = train_test_split(
            train_df, test_size=0.2, random_state=42, stratify=train_df['instrumentID'])

        # Generators (labels are already encoded)
        train_gen = SingleFeatureDataGenerator(train_df, feature_col, BATCH_SIZE, shuffle=True, num_classes=num_classes)
        val_gen   = SingleFeatureDataGenerator(val_df,   feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)
        test_gen  = SingleFeatureDataGenerator(test_df,  feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)

        # Model
        model = create_simple_model(input_shape, num_classes, model_name=feature_type)
        model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[early_stopping])
        history_list.append(history.history)

        # Evaluation
        loss, acc = model.evaluate(test_gen)
        loss_list.append(loss)
        accuracy_list.append(acc)
        print(f"{feature_type} - Fold {fold+1} Test accuracy: {acc:.4f}")

        # Predictions & Reports
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = []
        for _, labels in test_gen:
            y_true.extend(np.argmax(labels, axis=1))
        y_true = np.array(y_true)

        report = classification_report(y_true, y_pred_classes, output_dict=True)
        classification_reports.append(report)
        conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
        confusion_matrices.append(conf_matrix)

        # Save model
        os.makedirs(f"models/{feature_type}", exist_ok=True)
        model.save(f"models/{feature_type}/model_fold{fold+1}.keras")

    # Save results
    results[feature_type] = {
        "accuracy_list": accuracy_list,
        "loss_list": loss_list,
        "histories": history_list,
        "classification_reports": classification_reports,
        "confusion_matrices": confusion_matrices,
    }

    with open(f"models/{feature_type}/results.json", "w") as f:
        json.dump(results[feature_type], f, indent=2)

print("\nAll training complete. Models and results saved in 'models/'")

Training features:   0%|          | 0/10 [00:00<?, ?it/s]


Training model for mel_spectrogram

--- Fold 1/5 ---
Epoch 1/200
44/44 [==============================] - 1s 9ms/step - loss: 29.9972 - accuracy: 0.4996 - val_loss: 13.2067 - val_accuracy: 0.4133
Epoch 2/200
44/44 [==============================] - 0s 7ms/step - loss: 3.8078 - accuracy: 0.8523 - val_loss: 10.0827 - val_accuracy: 0.5520
Epoch 3/200
44/44 [==============================] - 0s 7ms/step - loss: 1.0429 - accuracy: 0.9377 - val_loss: 6.4971 - val_accuracy: 0.6503
Epoch 4/200
44/44 [==============================] - 0s 7ms/step - loss: 0.6194 - accuracy: 0.9515 - val_loss: 4.8199 - val_accuracy: 0.6850
Epoch 5/200
44/44 [==============================] - 0s 7ms/step - loss: 0.5696 - accuracy: 0.9587 - val_loss: 3.5481 - val_accuracy: 0.8121
Epoch 6/200
44/44 [==============================] - 0s 7ms/step - loss: 0.5021 - accuracy: 0.9718 - val_loss: 3.6430 - val_accuracy: 0.7948
Epoch 7/200
44/44 [==============================] - 0s 7ms/step - loss: 0.7658 - accuracy: 0.958

Training features:  10%|█         | 1/10 [00:36<05:29, 36.56s/it]


Training model for mfcc

--- Fold 1/5 ---
Epoch 1/200
44/44 [==============================] - 1s 7ms/step - loss: 5.2933 - accuracy: 0.4330 - val_loss: 2.9589 - val_accuracy: 0.2919
Epoch 2/200
44/44 [==============================] - 0s 6ms/step - loss: 1.3840 - accuracy: 0.6293 - val_loss: 3.3758 - val_accuracy: 0.1416
Epoch 3/200
44/44 [==============================] - 0s 6ms/step - loss: 1.1272 - accuracy: 0.6488 - val_loss: 3.5039 - val_accuracy: 0.2110
Epoch 4/200
44/44 [==============================] - 0s 6ms/step - loss: 1.3403 - accuracy: 0.6474 - val_loss: 1.6552 - val_accuracy: 0.4480
Epoch 5/200
44/44 [==============================] - 0s 6ms/step - loss: 0.9190 - accuracy: 0.7212 - val_loss: 1.5826 - val_accuracy: 0.4220
Epoch 6/200
44/44 [==============================] - 0s 6ms/step - loss: 0.7923 - accuracy: 0.7364 - val_loss: 1.0721 - val_accuracy: 0.6445
Epoch 7/200
44/44 [==============================] - 0s 6ms/step - loss: 0.6072 - accuracy: 0.7915 - val_loss: 

Training features:  20%|██        | 2/10 [01:09<04:37, 34.70s/it]

...

44/44 [==============================] - 1s 7ms/step - loss: 2.3580 - accuracy: 0.1723 - val_loss: 2.1582 - val_accuracy: 0.1561
Epoch 2/200
44/44 [==============================] - 0s 6ms/step - loss: 1.9649 - accuracy: 0.2788 - val_loss: 2.1013 - val_accuracy: 0.1821
Epoch 3/200
44/44 [==============================] - 0s 6ms/step - loss: 1.8481 - accuracy: 0.3295 - val_loss: 2.0741 - val_accuracy: 0.2225
Epoch 4/200
44/44 [==============================] - 0s 6ms/step - loss: 1.7740 - accuracy: 0.3773 - val_loss: 2.0873 - val_accuracy: 0.2110
Epoch 5/200
44/44 [==============================] - 0s 6ms/step - loss: 1.7201 - accuracy: 0.3910 - val_loss: 2.1387 - val_accuracy: 0.2139
Epoch 6/200
44/44 [==============================] - 0s 6ms/step - loss: 1.6691 - accuracy: 0.4178 - val_loss: 2.2126 - val_accuracy: 0.1763
Epoch 7/200
44/44 [==============================] - 0s 6ms/step - loss: 1.6181 - accuracy: 0.4185 - val_loss: 2.2107 - val_accuracy: 0.1965
Epoch 8/200
44/44 [======

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


44/44 [==============================] - 1s 7ms/step - loss: 2.3748 - accuracy: 0.1809 - val_loss: 2.2253 - val_accuracy: 0.1214
Epoch 2/200
44/44 [==============================] - 0s 6ms/step - loss: 1.9960 - accuracy: 0.2641 - val_loss: 2.1821 - val_accuracy: 0.1821
Epoch 3/200
44/44 [==============================] - 0s 6ms/step - loss: 1.9227 - accuracy: 0.2923 - val_loss: 2.2093 - val_accuracy: 0.2168
Epoch 4/200
44/44 [==============================] - 0s 6ms/step - loss: 1.8496 - accuracy: 0.3300 - val_loss: 2.1409 - val_accuracy: 0.2225
Epoch 5/200
44/44 [==============================] - 0s 6ms/step - loss: 1.8090 - accuracy: 0.3300 - val_loss: 2.1132 - val_accuracy: 0.2659
Epoch 6/200
44/44 [==============================] - 0s 6ms/step - loss: 1.7558 - accuracy: 0.3575 - val_loss: 2.0621 - val_accuracy: 0.2543
Epoch 7/200
44/44 [==============================] - 0s 6ms/step - loss: 1.7171 - accuracy: 0.3669 - val_loss: 2.1793 - val_accuracy: 0.2168
Epoch 8/200
44/44 [======

Training features: 100%|██████████| 10/10 [09:35<00:00, 57.57s/it]


All training complete. Models and results saved in 'models/'


In [15]:
print("\n" + "="*50)
print("Creating Ensemble Predictions")
print("="*50)

processed_df['instrumentID'] = processed_df['instrumentID'] - 1

ensemble_accuracies = []
ensemble_reports = []
ensemble_conf_matrices = []

for fold in range(KFOLD_SPLITS):
    print(f"\n--- Ensemble Fold {fold + 1}/{KFOLD_SPLITS} ---")
    fold_preds = []
    y_true = None

    for feature_type in FEATURE_TYPES:
        # Load model for this fold
        model_path = f"models/{feature_type}/model_fold{fold+1}.keras"
        if not os.path.exists(model_path):
            print(f"Model not found: {model_path}")
            continue
        model = keras.models.load_model(model_path)

        # Get test data for this fold
        feature_df = processed_df[[feature_type, 'instrumentID']].dropna().reset_index(drop=True)
        kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
        train_idx, test_idx = list(kf.split(feature_df))[fold]
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)
        test_gen = SingleFeatureDataGenerator(test_df, feature_type, batch_size=BATCH_SIZE, shuffle=False, num_classes=num_classes)
        preds = model.predict(test_gen, verbose=0)
        fold_preds.append(preds)
        if y_true is None:
            # Get true labels from generator
            y_true = []
            for _, labels in test_gen:
                y_true.extend(np.argmax(labels, axis=1))
            y_true = np.array(y_true)

    if fold_preds:
        ensemble_pred = np.mean(fold_preds, axis=0)
        ensemble_pred_classes = np.argmax(ensemble_pred, axis=1)
        acc = accuracy_score(y_true, ensemble_pred_classes)
        ensemble_accuracies.append(acc)
        print(f"Ensemble Accuracy: {acc:.4f}")
        report = classification_report(y_true, ensemble_pred_classes, output_dict=True)
        ensemble_reports.append(report)
        conf_matrix = confusion_matrix(y_true, ensemble_pred_classes).tolist()
        ensemble_conf_matrices.append(conf_matrix)


Creating Ensemble Predictions

--- Ensemble Fold 1/5 ---
Ensemble Accuracy: 0.8611

--- Ensemble Fold 2/5 ---
Ensemble Accuracy: 0.8449

--- Ensemble Fold 3/5 ---
Ensemble Accuracy: 0.8449

--- Ensemble Fold 4/5 ---
Ensemble Accuracy: 0.8634

--- Ensemble Fold 5/5 ---
Ensemble Accuracy: 0.8376


In [16]:
os.makedirs("ensemble_results", exist_ok=True)
date_part = datetime.now().date().__str__().replace('-', '_')
results_path = os.path.join("ensemble_results", f"ensemble_results_{date_part}.json")
ensemble_results = {
    "accuracy_list": ensemble_accuracies,
    "classification_reports": ensemble_reports,
    "confusion_matrices": ensemble_conf_matrices,
}
with open(results_path, "w") as f:
    json.dump(ensemble_results, f, indent=2)
print(f"\nEnsemble results saved to: {results_path}")




Ensemble results saved to: ensemble_results/ensemble_results_2025_07_08.json


In [17]:
for fold in range(KFOLD_SPLITS):
    models_fold = []
    inputs = []
    input_names = []
    for feature_type in FEATURE_TYPES:
        model_path = f"models/{feature_type}/model_fold{fold+1}.keras"
        if not os.path.exists(model_path):
            print(f"Model not found for ensemble: {model_path}")
            continue
        model = keras.models.load_model(model_path)
        models_fold.append(model)
        inp = model.input
        inputs.append(inp)
        input_names.append(feature_type)
    if not models_fold:
        print(f"No models found for fold {fold+1}, skipping ensemble model save.")
        continue
    # If all input shapes are the same, use a single input
    input_shapes = [tuple(inp.shape) for inp in inputs]
    if all(s == input_shapes[0] for s in input_shapes):
        ensemble_input = keras.Input(shape=input_shapes[0][1:], name="ensemble_input")
        model_outputs = [m(ensemble_input) for m in models_fold]
        avg = Average()(model_outputs)
        ensemble_model = Model(inputs=ensemble_input, outputs=avg, name=f"ensemble_model_fold{fold+1}")
    else:
        # Multi-input ensemble
        ensemble_inputs = [keras.Input(shape=inp.shape[1:], name=f"{name}_input") for inp, name in zip(inputs, input_names)]
        model_outputs = [m(inp) for m, inp in zip(models_fold, ensemble_inputs)]
        avg = Average()(model_outputs)
        ensemble_model = Model(inputs=ensemble_inputs, outputs=avg, name=f"ensemble_model_fold{fold+1}")
    # Save the ensemble model
    ensemble_model_path = os.path.join("ensemble_results", f"ensemble_model_fold{fold+1}_{date_part}.keras")
    ensemble_model.save(ensemble_model_path)
    print(f"Saved ensemble Keras model for fold {fold+1} to {ensemble_model_path}")


Saved ensemble Keras model for fold 1 to ensemble_results/ensemble_model_fold1_2025_07_08.keras
Saved ensemble Keras model for fold 2 to ensemble_results/ensemble_model_fold2_2025_07_08.keras
Saved ensemble Keras model for fold 3 to ensemble_results/ensemble_model_fold3_2025_07_08.keras
Saved ensemble Keras model for fold 4 to ensemble_results/ensemble_model_fold4_2025_07_08.keras
Saved ensemble Keras model for fold 5 to ensemble_results/ensemble_model_fold5_2025_07_08.keras


In [18]:
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\nIndividual Model Performance:")
for feature_type in FEATURE_TYPES:
    if feature_type in results:
        accuracies = results[feature_type]['accuracy_list']
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"  {feature_type}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\nEnsemble Performance:")
ensemble_mean = np.mean(ensemble_accuracies)
ensemble_std = np.std(ensemble_accuracies)
print(f"  Ensemble: {ensemble_mean:.4f} ± {ensemble_std:.4f}")

# Find best individual model
best_individual = max(
    [(ft, np.mean(results[ft]['accuracy_list'])) for ft in FEATURE_TYPES if ft in results],
    key=lambda x: x[1]
)
improvement = ensemble_mean - best_individual[1]
print(f"\nBest Individual Model: {best_individual[0]} ({best_individual[1]:.4f})")
print(f"Ensemble Improvement: {improvement:.4f} ({improvement*100:.2f}%)")


TRAINING SUMMARY

Individual Model Performance:
  mel_spectrogram: 0.8110 ± 0.0202
  mfcc: 0.6726 ± 0.0446
  chromagram: 0.3196 ± 0.0487
  spectral_contrast: 0.4025 ± 0.0371
  tonnetz: 0.3335 ± 0.0249
  constant_q: 0.6809 ± 0.0945
  cqt: 0.7651 ± 0.0352
  stft: 0.5095 ± 0.0685
  harmonic_percussive: 0.6156 ± 0.0352
  onset_strength: 0.1955 ± 0.0310

Ensemble Performance:
  Ensemble: 0.8504 ± 0.0101

Best Individual Model: mel_spectrogram (0.8110)
Ensemble Improvement: 0.0394 (3.94%)
